In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import os
import json
import numpy as np
from collections import defaultdict
from IPython.display import display, HTML
from collections import defaultdict

from matplotlib import colors as mcolors
from matplotlib import cm
from mycolorpy import colorlist as mcp
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

In [2]:
dirs = [
        "/home/noah/ESSL/experiments/ssl_testing3/BYOL",
        "/home/noah/ESSL/experiments/ssl_testing3/NNCLR",
        "/home/noah/ESSL/experiments/ssl_testing3/SimCLR",
        "/home/noah/ESSL/experiments/ssl_testing3/SwaV"
]


# vals = pd.read_csv(os.path.join(dirs[0], "outcomes.csv"))
vals = pd.DataFrame(columns=pd.read_csv(os.path.join(dirs[0], "outcomes.csv")).columns)
for d in dirs:
    f = os.path.join(d, "outcomes.csv")
    f = pd.read_csv(f)
    f["method"] = [os.path.basename(d) for _ in range(len(f))]
    vals = pd.concat([vals, f])


with open(os.path.join(dirs[0], "chromosomes.json"), "r") as f:
    chromos = json.load(f)
c_strings = []
ops = []
for i in range(len(chromos)):
    c = chromos[str(i)]
    c_strings.append(" ".join([c_i[0] for c_i in c]))
    c = [c_i[0] for c_i in c] + [c_i[1] for c_i in c]
    
    ops.append(c)
    
    
chromo_df = pd.DataFrame(ops, columns=["op1", "op2", "op3", "op1_i", "op2_i", "op3_i"])
# chromo_df
# chromo_colors = {chromo:color for chromo, color in zip(c_strings, colors)}
chromo_df["chromosome"] = c_strings
chromos = {}
for method in vals["method"].unique():
    vals_i = vals[vals["method"] == method]
    chromo_df_i = pd.concat([chromo_df, vals_i], axis=1)
    chromo_df_i = chromo_df_i.drop(columns=["Unnamed: 0"])
    chromos[method] = chromo_df_i
chromos

{'BYOL':             op1         op2         op3      op1_i      op2_i     op3_i  \
 0    Brightness      ShearX    Solarize   0.968611   0.004450  1.000000   
 1      Contrast      Rotate  TranslateX   0.161926   1.000000  5.000000   
 2        ShearY  TranslateY       Color   0.013367  13.000000  0.349641   
 3         Color    Solarize   Sharpness   0.610513   1.000000  0.922296   
 4        Rotate       Color  TranslateX  -1.000000   1.482108  3.000000   
 ..          ...         ...         ...        ...        ...       ...   
 240  TranslateY      ShearX   Sharpness  12.000000   0.123595  1.127105   
 241      Rotate    Contrast       Color   6.000000   1.498256  1.457496   
 242   Sharpness  TranslateX       Color   0.971129  10.000000  0.910740   
 243    Solarize  Brightness    Contrast   0.000000   1.555355  1.327163   
 244   Sharpness  TranslateX  Brightness   0.339779  11.000000  1.353143   
 
                           chromosome chromosome final ssl loss  \
 0         

In [3]:
all_ops = list(set(list(chromos['BYOL']["op1"].unique())+ list(chromos['BYOL']["op2"].unique()) + list(chromos['BYOL']["op3"].unique())))
for op in all_ops:
    print(op)

Brightness
Rotate
ShearY
Color
Solarize
TranslateX
Contrast
Sharpness
TranslateY
ShearX


In [4]:
indexes = {op:i for i, op in enumerate(all_ops)}
indexes
values = []
for method in chromos:
    for i, r in chromos[method].iterrows():
        ohe = np.empty(len(all_ops))
        ohe[:] = np.nan
        for op, i in zip(r[["op1", "op2", "op3"]], r[["op1_i", "op2_i", "op3_i"]]):
            ohe[indexes[op]] = i
        values.append(list(ohe) + [method, r["test acc"]])
dataset = pd.DataFrame(values, columns = all_ops + ["SSL algorithm", "test acc"])
dataset

,Brightness,Rotate,ShearY,Color,Solarize,TranslateX,Contrast,Sharpness,TranslateY,ShearX,SSL algorithm,test acc
0,0.968611,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.004450,BYOL,82.70
1,NaN,1.0,NaN,NaN,NaN,5.0,0.161926,NaN,NaN,NaN,BYOL,83.46
2,NaN,NaN,0.013367,0.349641,NaN,NaN,NaN,NaN,13.0,NaN,BYOL,82.73
3,NaN,NaN,NaN,0.610513,1.0,NaN,NaN,0.922296,NaN,NaN,BYOL,82.35
4,NaN,-1.0,NaN,1.482108,NaN,3.0,NaN,NaN,NaN,NaN,BYOL,82.80
...,...,...,...,...,...,...,...,...,...,...,...,...
975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.127105,12.0,0.123595,SwaV,82.53
976,NaN,6.0,NaN,1.457496,NaN,NaN,1.498256,NaN,NaN,NaN,SwaV,81.88
977,NaN,NaN,NaN,0.910740,NaN,10.0,NaN,0.971129,NaN,NaN,SwaV,82.38
978,1.555355,NaN,NaN,NaN,0.0,NaN,1.327163,NaN,NaN,NaN,SwaV,83.27


Get best and worst for BYOL for bootstrap exp

In [13]:
dataset[dataset["SSL algorithm"] == "BYOL"].sort_values(by="test acc")

best = dataset[dataset["SSL algorithm"] == "BYOL"].sort_values(by="test acc").iloc[-1]
worst = dataset[dataset["SSL algorithm"] == "BYOL"].sort_values(by="test acc").iloc[0]
print(best[best.notnull()])
print(worst)

Brightness       0.308851
Solarize              1.0
TranslateX           14.0
SSL algorithm        BYOL
test acc            83.96
Name: 132, dtype: object
Brightness            NaN
Rotate               19.0
ShearY                NaN
Color                 NaN
Solarize              NaN
TranslateX           11.0
Contrast         0.955471
Sharpness             NaN
TranslateY            NaN
ShearX                NaN
SSL algorithm        BYOL
test acc            81.92
Name: 159, dtype: object


In [ ]:
best = [["Brightness",0.308851],
        ["Solarize",1.0],
        ["TranslateX",14.0]]

worst = [["Rotate",19.0],
        ["TranslateX", 11.0],
        ["Contrast", 0.955471]]

